In [1]:
import re
from datetime import datetime
import math
import webbrowser
from urllib.parse import urlencode
import os
import serial
import time

def parse_serial_data(serial_port, baud_rate=115200, timeout=1, receive_time=5):
    """从串口读取经纬度数据，固定接收指定时间"""
    try:
        ser = serial.Serial(serial_port, baud_rate, timeout=timeout)
        print(f"成功打开串口: {serial_port}, 波特率: {baud_rate}")
        print(f"开始接收数据，将持续 {receive_time} 秒...")
        
        coordinates = []
        start_time = time.time()
        raw_data_buffer = ""
        
        try:
            buffer = ""
            
            while time.time() - start_time < receive_time:
                data = ser.read(ser.in_waiting or 1).decode('utf-8', errors='ignore')
                if not data:
                    continue
                    
                raw_data_buffer += data
                buffer += data
                
                # 打印原始数据片段
                if data:
                    print(f"[原始数据] {data.strip() or '<空>'}")
                
                # 匹配格式: 纬度,经度 或 纬度,经度
                matches = re.findall(r'?([\d.]+),([\d.]+)', buffer)
                
                for lat_str, lng_str in matches:
                    try:
                        lat = float(lat_str)
                        lng = float(lng_str)
                        
                        # 恢复经纬度校验
                        if -90 <= lat <= 90 and -180 <= lng <= 180:
                            coordinates.append((lng, lat))
                            print(f"[解析成功] WGS84坐标: 经度={lng:.6f}, 纬度={lat:.6f}")
                        else:
                            print(f"[范围错误] 经纬度超出范围: 经度={lng}, 纬度={lat}")
                    except Exception as e:
                        print(f"[解析失败] 错误: {e}, 原始数据: {lat_str},{lng_str}")
                
                # 从缓冲区移除已处理的数据
                if matches:
                    last_match_end = buffer.rfind(matches[-1][1]) + len(matches[-1][1])
                    buffer = buffer[last_match_end:]
                
                time.sleep(0.01)  # 减轻CPU负担
                
            print(f"\n数据接收完成，共耗时 {time.time() - start_time:.2f} 秒")
            print(f"[调试] 原始数据总长度: {len(raw_data_buffer)} 字符")
            print(f"[调试] 原始数据预览: {raw_data_buffer[:100]}...")
            
        except KeyboardInterrupt:
            print("\n用户中断，停止接收数据")
        finally:
            ser.close()
        
        return coordinates
    
    except serial.SerialException as e:
        print(f"串口打开失败: {e}")
        return []
    except Exception as e:
        print(f"接收数据时出错: {e}")
        return []

def print_received_data(coordinates):
    """打印所有接收到的WGS84坐标数据"""
    if not coordinates:
        print("没有接收到有效坐标数据")
        return
    
    print("\n===== 接收到的WGS84坐标数据 =====")
    for i, (lng, lat) in enumerate(coordinates):
        print(f"{i+1}. WGS84坐标: 经度={lng:.6f}, 纬度={lat:.6f}")
    print(f"===== 共 {len(coordinates)} 个坐标点 =====")

def display_coordinates(coordinates):
    """显示接收到的WGS84坐标数据摘要"""
    if not coordinates:
        print("没有接收到有效坐标数据")
        return []
    
    print("\n===== 接收到的WGS84坐标数据摘要 =====")
    
    for i, (lng, lat) in enumerate(coordinates):
        print(f"--- 坐标 {i+1} ---")
        print(f"  WGS84坐标: 经度={lng:.6f}, 纬度={lat:.6f}")
    
    print(f"\n===== 接收完成，共{len(coordinates)}个坐标点 =====")
    return coordinates

def convert_to_gcj02(coordinates):
    """将WGS84坐标转换为GCJ02坐标"""
    gcj_coordinates = []
    for lng, lat in coordinates:
        gcj_lng, gcj_lat = wgs84togcj02(lng, lat)
        gcj_coordinates.append((gcj_lng, gcj_lat))
        print(f"[转换完成] GCJ02坐标: 经度={gcj_lng:.6f}, 纬度={gcj_lat:.6f}")
    return gcj_coordinates

def create_amap_html(coordinates, title="实时轨迹地图", output_file="route——map.html"):
    """创建高德地图HTML文件，显示轨迹（使用GCJ02坐标）"""
    if not coordinates:
        print("没有有效的坐标数据，无法生成地图")
        return None
    
    points_str = ",\n        ".join([f"[{lng}, {lat}]" for lng, lat in coordinates])
    default_center = [116.403988, 39.91512]
    center_lng = sum(lng for lng, lat in coordinates) / len(coordinates) if coordinates else default_center[0]
    center_lat = sum(lat for lng, lat in coordinates) / len(coordinates) if coordinates else default_center[1]
    
    html_content = f"""<!DOCTYPE html>
<html>
<head>
    <meta charset="utf-8">
    <title>{title}</title>
    <script src="https://webapi.amap.com/maps?v=2.0&key={AMAP_API_KEY}"></script>
    <style>
        html, body, #container {{ width: 100%; height: 800px; margin: 0; padding: 0; }}
    </style>
</head>
<body>
    <div id="container"></div>
    <script type="text/javascript">
        var map = new AMap.Map('container', {{
            zoom: 13,
            center: [{center_lng}, {center_lat}]
        }});
        
        var path = [{points_str}];
        var polyline = new AMap.Polyline({{
            path: path,
            strokeColor: "#FF33FF",
            strokeWeight: 5,
            strokeOpacity: 0.8,
            lineJoin: 'round',
            lineCap: 'round',
            zIndex: 50,
            geodesic: true
        }});
        polyline.setMap(map);
        
        var startPoint = path[0];
        var endPoint = path[path.length - 1];
        
        new AMap.Marker({{
            position: startPoint,
            icon: "https://webapi.amap.com/theme/v1.3/markers/n/mark_b.png",
            offset: new AMap.Pixel(-10, -34)
        }}).setMap(map);
        
        new AMap.Marker({{
            position: endPoint,
            icon: "https://webapi.amap.com/theme/v1.3/markers/n/mark_r.png",
            offset: new AMap.Pixel(-10, -34)
        }}).setMap(map);
        
        map.setFitView();
    </script>
</body>
</html>"""
    
    with open(output_file, 'w', encoding='utf-8') as f:
        f.write(html_content)
    
    return output_file

def show_realtime_track(coordinates, title="实时轨迹地图", output_file="realtime_map.html"):
    """显示实时轨迹地图（使用GCJ02坐标）"""
    if not coordinates:
        print("没有有效的坐标数据，无法生成地图")
        return
    
    # 转换坐标
    gcj_coordinates = convert_to_gcj02(coordinates)
    
    # 生成地图
    map_file = create_amap_html(gcj_coordinates, title, output_file)
    if map_file:
        # 使用绝对路径打开
        file_url = f"file:///{os.path.abspath(map_file)}"
        webbrowser.open_new_tab(file_url)
        print(f"已生成实时轨迹地图: {map_file}")
        print(f"轨迹包含 {len(gcj_coordinates)} 个GCJ02坐标点")
        print(f"起点GCJ02坐标: 经度 {gcj_coordinates[0][0]:.6f}, 纬度 {gcj_coordinates[0][1]:.6f}")
        print(f"终点GCJ02坐标: 经度 {gcj_coordinates[-1][0]:.6f}, 纬度 {gcj_coordinates[-1][1]:.6f}")

def main():
    # 配置串口（根据实际情况修改）
    serial_port = "COM6"  # Windows串口示例，Linux/Mac可能为 "/dev/ttyUSB0"
    baud_rate = 115200
    receive_duration = 5  # 数据接收持续时间（秒）
    
    print("=== 高德地图实时轨迹绘制程序 ===")
    print("请确保串口已连接并发送WGS84格式经纬度数据")
    
    # 从串口接收数据
    gps_coordinates = parse_serial_data(serial_port, baud_rate, receive_time=receive_duration)
    
    if gps_coordinates:
        # 打印WGS84坐标数据
        print_received_data(gps_coordinates)
        
        # 显示WGS84坐标摘要
        coordinates = display_coordinates(gps_coordinates)
        
        # 生成并显示地图（自动转换为GCJ02坐标）
        show_realtime_track(coordinates, "WGS84转GCJ02轨迹地图")
    else:
        print("\n[错误] 未接收到有效坐标数据，请检查：")
        print("1. 串口是否正确连接（端口号：COM9）")
        print("2. 波特率是否匹配（115200）")
        print("3. 数据格式是否为：纬度,经度 或 纬度,经度")

if __name__ == "__main__":
    main()

=== 高德地图实时轨迹绘制程序 ===
请确保串口已连接并发送WGS84格式经纬度数据
成功打开串口: COM6, 波特率: 115200
开始接收数据，将持续 5 秒...
[原始数据] 2
[原始数据] 9.715706,106.784225
       29.715702,106.784225
       29.715702,106.784248
       29.715702,106.784248
       29.715702,106.784264
       29.715702,106.784264
       29.715702,106.784279
       29.715702,106.784294
       29.715706,106.784309
       29.715706,106.784325
       29.715702,106.784340
       29.715698,106.784355
       29.715702,106.784378
       29.715702,106.784393
       
[解析成功] WGS84坐标: 经度=106.784225, 纬度=29.715706
[解析成功] WGS84坐标: 经度=106.784225, 纬度=29.715702
[解析成功] WGS84坐标: 经度=106.784248, 纬度=29.715702
[解析成功] WGS84坐标: 经度=106.784248, 纬度=29.715702
[解析成功] WGS84坐标: 经度=106.784264, 纬度=29.715702
[解析成功] WGS84坐标: 经度=106.784264, 纬度=29.715702
[解析成功] WGS84坐标: 经度=106.784279, 纬度=29.715702
[解析成功] WGS84坐标: 经度=106.784294, 纬度=29.715702
[解析成功] WGS84坐标: 经度=106.784309, 纬度=29.715706
[解析成功] WGS84坐标: 经度=106.784325, 纬度=29.715706
[解析成功] WGS84坐标: 经度=106.784340, 纬度=29.715702
[解析成功